In [6]:
!SET SKLEARN_ALLOW_DEPRECATED_SKLEARN_PACKAGE_INSTALL=True

In [9]:
!pip install pandas numpy scikit-learn nltk langdetect

  Using cached pandas-2.3.1-cp312-cp312-win_amd64.whl.metadata (19 kB)
   ---------------------------------------- 0.0/11.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/11.0 MB ? eta -:--:--
    --------------------------------------- 0.3/11.0 MB ? eta -:--:--
    --------------------------------------- 0.3/11.0 MB ? eta -:--:--
   - -------------------------------------- 0.5/11.0 MB 762.0 kB/s eta 0:00:14
   - -------------------------------------- 0.5/11.0 MB 762.0 kB/s eta 0:00:14
   -- ------------------------------------- 0.8/11.0 MB 745.8 kB/s eta 0:00:14
   --- ------------------------------------ 1.0/11.0 MB 774.8 kB/s eta 0:00:13
   --- ------------------------------------ 1.0/11.0 MB 774.8 kB/s eta 0:00:13
   ---- ----------------------------------- 1.3/11.0 MB 780.2 kB/s eta 0:00:13
   ---- ----------------------------------- 1.3/11.0 MB 780.2 kB/s eta 0:00:13
   ----- ---------------------------------- 1.6/11.0 MB 735.9 kB/s eta 0:00:13
   ----- --------

In [10]:
import pickle
import os

In [11]:
train = "wikilingua_multilingual.train.pk"
val = "wikilingua_multilingual.val.pk"
test = "wikilingua_multilingual.test.pk"

data_folder = os.path.join("Dataset")

train_path = os.path.join(data_folder, train)
val_path = os.path.join(data_folder, val)
test_path = os.path.join(data_folder, test)

# Read & change language code

In [12]:
import pandas as pd
lang_map = {
    'en': 'English',
    'es': 'Spanish',
    'pt': 'Portuguese',
    'fr': 'French',
    'de': 'German',
    'ru': 'Russian',
    'it': 'Italian',
    'id': 'Indonesian',
    'nl': 'Dutch',
    'ar': 'Arabic',
    'vi': 'Vietnamese',
    'zh': 'Chinese',
    'th': 'Thai',
    'ja': 'Japanese',
    'ko': 'Korean',
    'hi': 'Hindi',
    'cs': 'Czech',
    'tr': 'Turkish',
}

In [19]:
def read_data(path):
    with open(path, "rb") as f:
        data = pickle.load(f)
    df = pd.DataFrame(data)
    df.drop(['target', 'id'], axis=1, inplace=True)
    df['source'] = df['source'].map(lang_map)
    df = df[~df['source'].isin(['Thai'])]
    return df

In [20]:
train_dts = read_data(train_path)
val_dts = read_data(val_path)
test_dts = read_data(test_path)

In [21]:
train_dts['source'].value_counts()

source
English       95517
Spanish       76295
Portuguese    54673
French        43423
German        39505
Russian       35313
Italian       34085
Indonesian    32228
Dutch         21345
Arabic        19992
Vietnamese    13262
Chinese       12523
Japanese       8657
Korean         8370
Hindi          6724
Czech          4872
Turkish        3052
Name: count, dtype: int64

# Save dts for abstractive model

In [22]:
abtractive_dts = {
    'train': train_dts,
    'validation': val_dts,
    'test': test_dts
}

with open(os.path.join(data_folder, "abstractive", "abstractive.pk"), "wb") as file:
    pickle.dump(abtractive_dts, file)

# Save dts for extractive model

In [15]:
from langdetect import detect
import nltk
# from bltk.langtools import Tokenizer
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\GIGABYTE\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [23]:
def split_sentence(text):
    try:
        lang = detect(text)
    except:
        return [text]

    if lang in ['en', 'es', 'fr', 'de', 'it', 'pt', 'nl']:  # English, Spanish, French, German, etc.
        return nltk.sent_tokenize(text, language=lang)

    elif lang == 'vi':
        return [s.strip() for s in text.replace("!", ".").replace("?", ".").split(".") if s.strip()]

    elif lang == 'zh': # Chinese
        import re
        return re.split(r'(。|！|\!|\.|？|\?)', text)

    elif lang == 'ar': # Arabic
        return [s.strip() for s in text.replace("؟", ".").replace("!", ".").split(".") if s.strip()]

    else: # Defautl
        return [s.strip() for s in text.replace("!", ".").replace("?", ".").split(".") if s.strip()]

In [30]:
text = train_dts.iloc[4, 0]
sentences = split_sentence(text)
sentences[:5]

['Masaj, kan akışını hızlandırabilir ve dudaklarını aydınlatabilir',
 'Her gece yatmadan önce dudaklarını derinlemesine nemlendirmek için dudaklarına masaj yap',
 'Alternatif olarak, dudaklarını nemlendirmek ve daha dolgun ve pembe görünmelerini sağlamak için buz küpleriyle masaj yap',
 'Dudaklarını soymanın çeşitli yolları vardır',
 'Cilt soyucu ürün kullanmayı tercih edebilirsin ya da dudaklarını bir diş fırçasıyla ovalayabilirsin']